<a href="https://colab.research.google.com/github/Tian1398/IMG542-Project/blob/main/542NeworkProject_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import mailbox
import email
import json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
inbox_mbox_path = 'drive/My Drive/574/Inbox.mbox'
sent_mbox_path = 'drive/My Drive/574/Sent.mbox'

In [20]:
# Read the mbox files
inbox_mbox = mailbox.mbox(inbox_mbox_path)
sent_mbox = mailbox.mbox(sent_mbox_path)

# Create empty lists to store the extracted data
inbox_data = []
sent_data = []

In [21]:
# Headers to extract
headers_to_extract = ['Date', 'From', 'To', 'Subject']

In [ ]:
# Iterate through each message in the mbox file
for message in inbox_mbox:
    try:
        msg = email.message_from_bytes(message.as_bytes())
        email_data = {header: msg.get(header, None) for header in headers_to_extract}
        inbox_data.append(email_data)
    except Exception as e:
        print(f"Error processing message: {e}")

# Create a DataFrame
inbox_df = pd.DataFrame(inbox_data, columns=headers_to_extract)

# clean data: remove null
inbox_df = inbox_df.dropna(subset=['To'])

In [30]:
# Iterate through each message in the Sent mbox file
for message in sent_mbox:
    try:
        msg = email.message_from_bytes(message.as_bytes())
        email_data = {header: msg.get(header, None) for header in headers_to_extract}
        sent_data.append(email_data)
    except Exception as e:
        print(f"Error processing message: {e}")

# Create a DataFrame
sent_df = pd.DataFrame(sent_data, columns=headers_to_extract)
sent_df = sent_df.dropna(subset=['To'])

,Date,From,To,Subject
0,"Mon, 22 Apr 2024 16:31:58 -0700",Chengzi Tian <tiancz23@uw.edu>,Chengzi Tian <tiancz23@uw.edu>,
1,"Sun, 3 Mar 2024 23:58:28 -0800",Chengzi Tian <tiancz23@uw.edu>,tiancz23 <tiancz23@uw.edu>,Draft
2,"Mon, 19 Feb 2024 19:44:09 -0800",Chengzi Tian <tiancz23@uw.edu>,hr@itsuggest.com,Application for Business System/Intelligence A...
3,"Tue, 23 Apr 2024 17:01:28 -0700",Chengzi Tian <tiancz23@uw.edu>,Chengzi Tian <tiancz23@uw.edu>,resume
4,"Fri, 9 Feb 2024 12:38:13 -0800",Chengzi Tian <tiancz23@uw.edu>,"""Felts, Alexandria"" <alexandria.felts@howard.edu>",Re: Tian - Interview - JR104905 Business Intel...


In [69]:
# Combine datasets
email_df = pd.concat([inbox_df, sent_df], ignore_index=True)
email_df.shape

(3622, 4)

# expand multi recipient edges

In [72]:
# define a function to expand data by "To" column
def split_and_expand(row):
    to_addresses = row['To'].split('>,')
    expanded_rows = []
    for address in to_addresses:
        cleaned_address = address.strip().strip('<').strip()
        expanded_rows.append({
            'Date': row['Date'],
            'From': row['From'],
            'To': cleaned_address,
            'Subject': row['Subject']})

    return expanded_rows

In [71]:
single_way = inbox_df[~inbox_df['To'].str.contains(">,")]
multi_way = email_df[email_df['To'].str.contains(">,")]

In [73]:
# call functino to expand data
expanded_rows = []
for _, row in multi_way.iterrows():
    expanded_rows.extend(split_and_expand(row))

expanded_multi_df = pd.DataFrame(expanded_rows)

In [74]:
expand_email_df = pd.concat([single_way, expanded_multi_df], ignore_index=True)
expand_email_df.shape

(5516, 4)

In [75]:
expand_email_df.head(20)

,Date,From,To,Subject
0,"Thu, 23 May 2024 22:32:24 +0000",iSchool Career Services <notifications@instruc...,tiancz23@uw.edu,Your Weekly Job Round-Up!: iSchool Career Serv...
1,"Tue, 21 May 2024 19:13:33 +0000",Handshake <handshake@g.joinhandshake.com>,tiancz23@uw.edu,You're invited to a virtual event with Handshake
2,"Mon, 13 May 2024 21:30:23 +0000",MSIM Academic Advising <iSchoolStudentServices...,Chengzi Tian <tiancz23@uw.edu>,Reminder: Early Career Accelerated - Virtual A...
3,"Mon, 13 May 2024 11:58:28 +0000","""Indeed"" <donotreply@indeed.com>",<tiancz23@uw.edu>,Financial Analyst @ Atlantic Group
4,"Sun, 26 May 2024 07:00:50 +0000 (UTC)",Ford Careers <noreplyr@ford.com>,tiancz23@uw.edu,"Confirm your identity for job Analyst, Digital..."
5,"Fri, 24 May 2024 21:09:21 +0000",Ryan Hursh via stlp_updates <stlp_updates@u.wa...,"""stlp_updates@uw.edu"" <stlp_updates@uw.edu>",[STLP Updates] STLP Equipment Return
6,"Wed, 08 May 2024 06:09:10 +0000","""Indeed"" <donotreply@indeed.com>",<tiancz23@uw.edu>,PROGRAM MANAGER FULFILLMENT SERVICES @ DCG ONE
7,"Tue, 14 May 2024 22:45:39 +0000","""Indeed"" <donotreply@indeed.com>",<tiancz23@uw.edu>,Master Data Analyst @ Integrated Resources - R...
8,"Mon, 27 May 2024 07:07:59 -0600 (MDT)",Ford Candidate Experience <noreplycandex@qualt...,Chengzi Tian <tiancz23@uw.edu>,Ford Motor Company - Application Experience
9,"Tue, 28 May 2024 06:14:52 +0000",DrFirst Assessment Center <DO-NOT-REPLY@criter...,tiancz23@uw.edu,Thank you


# calculate frenquncy

In [92]:
df1 = expand_email_df.iloc[:, 1:3]

In [93]:
df1['frequency'] = df1.groupby(['From', 'To'])['From'].transform('count')

<ipython-input-93-dc82fb442d7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['frequency'] = df1.groupby(['From', 'To'])['From'].transform('count')


# get unique names

In [94]:
# Get unique values from "From" and "To" columns
unique_from = df1['From'].unique()
unique_to = df1['To'].unique()

# Concatenate unique values into a new DataFrame
df2 = pd.concat([pd.DataFrame(unique_from, columns=['Unique']),
                              pd.DataFrame(unique_to, columns=['Unique'])], ignore_index=True)

# Remove duplicate values if any
df2.drop_duplicates(inplace=True)

In [95]:
df2

,Unique
0,iSchool Career Services <notifications@instruc...
1,Handshake <handshake@g.joinhandshake.com>
2,MSIM Academic Advising <iSchoolStudentServices...
3,"""Indeed"" <donotreply@indeed.com>"
4,Ford Careers <noreplyr@ford.com>
...,...
1492,"""IMAMuse@uw.edu""\n\t<IMAMuse@uw.edu"
1493,Robert Lehman\n\t<rlehman1@uw.edu
1494,Amy Wang\n\t<amywangw@uw.edu
1495,iphds <iphds@uw.edu>


In [79]:
test.replace("tiancz23@uw.edu", "me", inplace=True)

<ipython-input-79-3109374afc75>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.replace("tiancz23@uw.edu", "me", inplace=True)


In [90]:
test.dtypes

From         object
To           object
frequency     int64
dtype: object

In [91]:
test["From"].astype(str)
test["To"].astype(str)

0                                               me
1                                               me
2                   Chengzi Tian <tiancz23@uw.edu>
3                                <tiancz23@uw.edu>
4                                               me
                           ...                    
5511    "Lucylin03@gmail.com" <Lucylin03@gmail.com
5512                  Linh Pham\n\t<lpham21@uw.edu
5513                 Nassim Parvin <nassimi@uw.edu
5514              Menita Agarwal\n\t<menita@uw.edu
5515           Isha Doshi <ishardoshi@outlook.com>
Name: To, Length: 5516, dtype: object

In [86]:
#test = test.apply(lambda x: x.str.replace(".*tiancz23.*", "me", regex=True))
# Replace all cells containing "tian" with "me"
test[test.apply(lambda x: x.str.contains("tiancz23")).any(axis=1)] = 'me'

AttributeError: Can only use .str accessor with string values!

# download data

In [96]:
# Create a Pandas Excel writer using ExcelWriter
with pd.ExcelWriter('output.xlsx') as writer:
    # Write each DataFrame to a separate sheet
    df2.to_excel(writer, sheet_name='nodes', index=False)  # Writing df2 to Sheet1 without index
    df1.to_excel(writer, sheet_name='edges', index=False)  # Writing df1 to Sheet2 without index

In [97]:
from google.colab import files

# Assuming you have already saved the Excel file
files.download('output.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Below are drafts

In [65]:
test2 = sent_df[sent_df['To'].str.contains(">,")]

In [66]:
expanded_rows2 = []
for _, row in test2.iterrows():
    expanded_rows2.extend(split_and_expand(row))

expanded_df2 = pd.DataFrame(expanded_rows2) # 21
expanded_df2

In [55]:
# multipl recipients emails in inbox
test = inbox_df[inbox_df['To'].str.contains(">,")]

In [56]:
def split_and_expand(row):
    to_addresses = row['To'].split('>,')
    expanded_rows = []
    for address in to_addresses:
        cleaned_address = address.strip().strip('<').strip()
        expanded_rows.append({
            'Date': row['Date'],
            'From': row['From'],
            'To': cleaned_address,
            'Subject': row['Subject']})

    return expanded_rows

# Expand the DataFrame
expanded_rows = []
for _, row in test.iterrows():
    expanded_rows.extend(split_and_expand(row))

expanded_df = pd.DataFrame(expanded_rows)

In [57]:
expanded_df

,Date,From,To,Subject
0,"Thu, 18 Apr 2024 19:43:04 +0000",Association of Information Mgmt Students via i...,"""imsim@uw.edu"" <imsim@uw.edu",[iMSIM] MSIM Spring Quarter Highlights: Events...
1,"Thu, 18 Apr 2024 19:43:04 +0000",Association of Information Mgmt Students via i...,"""imsimonline@uw.edu"" <imsimonline@uw.edu>",[iMSIM] MSIM Spring Quarter Highlights: Events...
2,"Tue, 7 May 2024 19:58:57 +0000",ASSOCIATION OF LIBRARY AND INFORMATION SCIENCE...,"""MLIS Listserv "" <imlis@uw.edu",[iMSIM] iGala Reminders
3,"Tue, 7 May 2024 19:58:57 +0000",ASSOCIATION OF LIBRARY AND INFORMATION SCIENCE...,"""imsim@uw.edu"" <imsim@uw.edu",[iMSIM] iGala Reminders
4,"Tue, 7 May 2024 19:58:57 +0000",ASSOCIATION OF LIBRARY AND INFORMATION SCIENCE...,"""iMAMuse@uw.edu"" <iMAMuse@uw.edu>",[iMSIM] iGala Reminders
...,...,...,...,...
2391,"Wed, 1 May 2024 18:30:00 +0000",iSchool Affiliates via iMSIM <imsim@u.washingt...,"""imsim@uw.edu"" <imsim@uw.edu",[iMSIM] Sign Up Now | iAffiliates Coalfire Pan...
2392,"Wed, 1 May 2024 18:30:00 +0000",iSchool Affiliates via iMSIM <imsim@u.washingt...,"""iphds@uw.edu"" <iphds@uw.edu",[iMSIM] Sign Up Now | iAffiliates Coalfire Pan...
2393,"Wed, 1 May 2024 18:30:00 +0000",iSchool Affiliates via iMSIM <imsim@u.washingt...,"""imsimonline@uw.edu"" <imsimonline@uw.edu",[iMSIM] Sign Up Now | iAffiliates Coalfire Pan...
2394,"Wed, 1 May 2024 18:30:00 +0000",iSchool Affiliates via iMSIM <imsim@u.washingt...,"""imlis@uw.edu"" <imlis@uw.edu",[iMSIM] Sign Up Now | iAffiliates Coalfire Pan...


In [38]:
inbox_df[~inbox_df['To'].str.contains("tiancz23@uw.edu", na=False)]
# if the column "To" contains None (missing) value

,Date,From,To,Subject
5,"Fri, 24 May 2024 21:09:21 +0000",Ryan Hursh via stlp_updates <stlp_updates@u.wa...,"""stlp_updates@uw.edu"" <stlp_updates@uw.edu>",[STLP Updates] STLP Equipment Return
12,"Thu, 16 May 2024 19:11:26 -0700",Chengzi Tian <tiancz23@uw.edu>,Heather Whiteman <iheather@uw.edu>,"Re: New Event: Chengzi Tian - 02:40pm Tue, May..."
17,"Mon, 22 Apr 2024 22:53:55 +0000",iSchool MSIM <msim@uw.edu>,None,Re: ECA Practicum Summer Quarter
18,"Fri, 12 Apr 2024 20:50:55 +0000",iSchool MSIM <msim@uw.edu>,None,ECA Practicum Summer Quarter
19,"Thu, 18 Apr 2024 19:43:04 +0000",Association of Information Mgmt Students via i...,"""imsim@uw.edu"" <imsim@uw.edu>, ""imsimonline@uw...",[iMSIM] MSIM Spring Quarter Highlights: Events...
...,...,...,...,...
3495,"Wed, 31 Jan 2024 08:40:51 -0800",Chengzi Tian <tiancz23@uw.edu>,Tanner Metzler <tanner@mks2.com>,Re: MKS2 - Let's Discuss your Application!
3497,"Sat, 6 Apr 2024 00:47:15 +0000",iSchool MSIM via iMSIM <imsim@u.washington.edu>,"""imsim@uw.edu"" <imsim@uw.edu>",[iMSIM] Registration Announcement: Summer 2024...
3500,"Thu, 9 May 2024 23:38:22 +0000",Cameron Cramer via iMSIM <imsim@u.washington.edu>,"""imsim@uw.edu"" <imsim@uw.edu>",[iMSIM] Opportunities & Events for Internation...
3505,"Fri, 17 May 2024 09:48:36 +0000",ASSOCIATION OF LIBRARY AND INFORMATION SCIENCE...,"""imsim@uw.edu"" <imsim@uw.edu>, ""iMAMuse@uw.edu...",[iMSIM] Book Sale Round 2


In [32]:
rows, columns = sent_df.shape
print(f"The sent_df has {rows} rows and {columns} columns.")

The sent_df has 149 rows and 4 columns.


In [24]:
rows, columns = inbox_df.shape
print(f"The DataFrame has {rows} rows and {columns} columns.")

The DataFrame has 3531 rows and 4 columns.


In [34]:
sent_df[sent_df['From'].isna()]

# cleaned_df = inbox_df.dropna(subset=['To'])

,Date,From,To,Subject


In [15]:
all_keys = set()

# Iterate through each message in the mbox file
for message in mbox:
    msg = email.message_from_bytes(message.as_bytes())
    all_keys.update(msg.keys())

# Print all unique headers (keys)
print("All unique headers found in emails:")
for key in sorted(all_keys):
    print(key)

All unique headers found in emails:
ARC-Authentication-Results
ARC-Message-Signature
ARC-Seal
Accept-Language
Authentication-Results
Authentication-Results-Original
Auto-Submitted
CC
Cc
Charset
Content-Class
Content-ID
Content-Id
Content-Language
Content-Transfer-Encoding
Content-Type
Content-transfer-encoding
Content-type
DKIM-Filter
DKIM-Signature
Date
Deferred-Delivery
Delivered-To
DomainKey-Signature
Errors-To
FROM
Feedback-ID
Feedback-Id
From
Importance
In-Reply-To
IronPort-Data
IronPort-HdrOrdr
List-Archive
List-Help
List-ID
List-Id
List-Post
List-Subscribe
List-Unsubscribe
List-Unsubscribe-Post
MIME-Version
MIME-version
Message-ID
Message-Id
Message-id
Mime-Version
Origin-messageId
Original-Envelope-Id
Precedence
Priority
RPCampaign
Received
Received-SPF
Recipient-Id
References
Reply-To
Reply-to
Return-Path
Return-Receipt-To
Sender
Sender-IP
SenderEmail
SenderName
Sensitivity
Site-Id
Subject
TM-MAIL-JID
TO
Thread-Index
Thread-Topic
To
User-Agent
X-250ok-CID
X-ASG-Debug-ID
X-ASG-

In [ ]:
# Read the mbox files
inbox_mbox = mailbox.mbox(inbox_mbox_path)

# List to store the extracted data
inbox_data = []

# Headers to extract
headers_to_extract = ['Date', 'From', 'To', 'Subject']

# Iterate through each message in the mbox file
for message in inbox_mbox:
    try:
        msg = email.message_from_bytes(message.as_bytes())
        email_data = {header: msg.get(header, None) for header in headers_to_extract}
        inbox_data.append(email_data)
    except Exception as e:
        print(f"Error processing message: {e}")

# Create a DataFrame
inbox_df = pd.DataFrame(inbox_data, columns=headers_to_extract)

inbox_df.head()

In [12]:
sent_mbox_path = 'drive/My Drive/574/Sent.mbox'

# Read the mbox file
sent = mailbox.mbox(sent_mbox_path)

# List to store the extracted data
data = []

# Headers to extract
headers_to_extract = ['Date', 'From', 'To', 'Subject']

# Iterate through each message in the mbox file
for message in sent:
    try:
        msg = email.message_from_bytes(message.as_bytes())
        email_data = {header: msg.get(header, None) for header in headers_to_extract}
        data.append(email_data)
    except Exception as e:
        print(f"Error processing message: {e}")

# Create a DataFrame
df = pd.DataFrame(data, columns=headers_to_extract)

df.head()

,Date,From,Sender,Subject,To
0,"Mon, 22 Apr 2024 16:31:58 -0700",Chengzi Tian <tiancz23@uw.edu>,None,,Chengzi Tian <tiancz23@uw.edu>
1,"Sun, 3 Mar 2024 23:58:28 -0800",Chengzi Tian <tiancz23@uw.edu>,None,Draft,tiancz23 <tiancz23@uw.edu>
2,"Mon, 19 Feb 2024 19:44:09 -0800",Chengzi Tian <tiancz23@uw.edu>,None,Application for Business System/Intelligence A...,hr@itsuggest.com
3,"Tue, 23 Apr 2024 17:01:28 -0700",Chengzi Tian <tiancz23@uw.edu>,None,resume,Chengzi Tian <tiancz23@uw.edu>
4,"Fri, 9 Feb 2024 12:38:13 -0800",Chengzi Tian <tiancz23@uw.edu>,None,Re: Tian - Interview - JR104905 Business Intel...,"""Felts, Alexandria"" <alexandria.felts@howard.edu>"


In [ ]:
# Function to convert email to JSON without attachments
def email_to_json(msg):
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == 'text/plain' and 'attachment' not in part.get('Content-Disposition', ''):
                {
        'from': msg['From'],
        'to': msg['To'],
        'subject': msg['Subject'],
        'date': msg['Date'],
        'body': None,
    }['body'] = part.get_payload(decode=True).decode(errors='ignore')
                break
    else:
        new_var = {
        'from': msg['From'],
        'to': msg['To'],
        'subject': msg['Subject'],
        'date': msg['Date'],
        'body': None,
    }
        new_var['body'] = msg.get_payload(decode=True).decode(errors='ignore')

    return {
        'from': msg['From'],
        'to': msg['To'],
        'subject': msg['Subject'],
        'date': msg['Date'],
        'body': None,
    }